### Create scenarios and upgrade congested transmission line

In [1]:
from powersimdata import Scenario
scenario = Scenario('')
scenario.set_builder(grid_model="usa_tamu", interconnect="Eastern")
scenario.set_name("MISONC_MTEP16_MEP", "training")
scenario.set_time("2016-01-01 00:00:00","2016-01-31 23:00:00","24H")
scenario.set_base_profile("demand", "vJan2021")
scenario.set_base_profile("hydro", "vJan2021")
scenario.set_base_profile("solar", "vJan2021")
scenario.set_base_profile("wind", "vJan2021")

# Upgrade the congested branch 68928 from 13.66MVA RateA to 236MW, which is a 17.28-time increase. Impedance will be automatically reduced proportionally.
scenario.change_table.scale_branch_capacity(branch_id = {68928: 17.28})

grid = scenario.get_grid()
ct = scenario.get_ct()

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
--> Summary
# Existing study
test
# Available profiles
demand: vJan2021
hydro: vJan2021
solar: vJan2021
wind: vJan2021


### Use the change table to modify conditions

In [2]:
scenario.change_table.scale_plant_capacity(
    "solar", zone_name={"Arkansas": 5, "Louisiana": 2.5})
scenario.change_table.scale_plant_capacity(
    "wind", zone_name={"Iowa": 1.5, "Minnesota Southern": 2})
scenario.change_table.scale_branch_capacity(
    zone_name={"Iowa": 2, "Minnesota Southern": 2})

# add AC line between two busses in IA
scenario.change_table.add_branch(
    [{"capacity": 236, "from_bus_id": 54070, "to_bus_id": 54057}])

# add DC line between central IA and Chicago, IL
scenario.change_table.add_dcline(
    [{"capacity": 2000, "from_bus_id":55119, "to_bus_id": 1213}])

# add a solar plant in AK, a coal plant in IA and a natural gas plant in LA
scenario.change_table.add_plant(
    [{"type": "solar", "bus_id": 1899, "Pmax": 75},
     {"type": "coal", "bus_id": 54070, "Pmin": 25, "Pmax": 750, "c0": 1800, "c1": 30, "c2": 0.0025},
     {"type": "ng", "bus_id": 60911, "Pmax": 75, "c0": 900, "c1": 30, "c2": 0.0015}])

# add a new bus, and a new one-way DC line connected to this bus
scenario.change_table.add_bus(
	[{"lat": 35, "lon": -90, "zone_id": 42, "baseKV": 500}])
scenario.change_table.add_dcline(
	[{"from_bus_id": 1899, "to_bus_id": 70048, "Pmin": 0, "Pmax": 200}])

### Review the scenario and change table, the launch simulation

In [3]:
grid = scenario.get_grid()
ct = scenario.get_ct()
scenario.print_scenario_info()

--------------------
SCENARIO INFORMATION
--------------------
plan: MISONC_MTEP16_MEP
name: training
state: create
grid_model: usa_tamu
interconnect: Eastern
base_demand: vJan2021
base_hydro: vJan2021
base_solar: vJan2021
base_wind: vJan2021
change_table: Yes
start_date: 2016-01-01 00:00:00
end_date: 2016-01-31 23:00:00
interval: 24H
engine: REISE.jl


In [4]:
ct

{'branch': {'branch_id': {68928: 17.28}, 'zone_id': {39: 2, 38: 2}},
 'solar': {'zone_id': {42: 5, 43: 2.5}},
 'wind': {'zone_id': {39: 1.5, 38: 2}},
 'new_branch': [{'from_bus_id': 54070,
   'to_bus_id': 54057,
   'Pmax': 236,
   'Pmin': -236}],
 'new_dcline': [{'from_bus_id': 55119,
   'to_bus_id': 1213,
   'Pmax': 2000,
   'Pmin': -2000},
  {'from_bus_id': 1899, 'to_bus_id': 70048, 'Pmin': 0, 'Pmax': 200}],
 'new_plant': [{'type': 'solar',
   'bus_id': 1899,
   'Pmax': 75,
   'Pmin': 0,
   'plant_id_neighbor': 674},
  {'type': 'coal',
   'bus_id': 54070,
   'Pmin': 25,
   'Pmax': 750,
   'c0': 1800,
   'c1': 30,
   'c2': 0.0025},
  {'type': 'ng',
   'bus_id': 60911,
   'Pmax': 75,
   'c0': 900,
   'c1': 30,
   'c2': 0.0015,
   'Pmin': 0}],
 'new_bus': [{'lat': 35, 'lon': -90, 'zone_id': 42, 'baseKV': 500, 'Pd': 0}]}

In [5]:
scenario.create_scenario()

print(scenario.state.name)
scenario.state.print_scenario_status()

CREATING SCENARIO: MISONC_MTEP16_MEP | training 

--> Adding entry in ScenarioList.csv on server
--> Writing change table on local machine
Writing /root/ScenarioData/405_ct.pkl
--> Moving file /root/ScenarioData/405_ct.pkl to /mnt/bes/pcm/data/input/405_ct.pkl
-->  Setting status=created in execute table on server
SCENARIO SUCCESSFULLY CREATED WITH ID #405
State switching: create --> execute
execute
---------------
SCENARIO STATUS
---------------
created


In [6]:
scenario.state.prepare_simulation_input()

---------------------------
PREPARING SIMULATION INPUTS
---------------------------
--> Creating temporary folder on server for simulation inputs
--> Loading demand
Writing scaled demand profile in /root/ScenarioData/ on local machine
--> Moving file /root/ScenarioData/405_demand.csv to /mnt/bes/pcm/tmp/scenario_405/demand.csv
--> Loading hydro
hydro_vJan2021.csv not found in /root/ScenarioData/ on local machine
--> Downloading hydro_vJan2021.csv from blob storage.


  0%|          | 0.00/222M [00:00<?, ?B/s]

Writing scaled hydro profile in /root/ScenarioData/ on local machine
--> Moving file /root/ScenarioData/405_hydro.csv to /mnt/bes/pcm/tmp/scenario_405/hydro.csv
--> Loading solar
Writing scaled solar profile in /root/ScenarioData/ on local machine
--> Moving file /root/ScenarioData/405_solar.csv to /mnt/bes/pcm/tmp/scenario_405/solar.csv
--> Loading wind
Writing scaled wind profile in /root/ScenarioData/ on local machine
--> Moving file /root/ScenarioData/405_wind.csv to /mnt/bes/pcm/tmp/scenario_405/wind.csv
Building MPC file
--> Moving file /root/ScenarioData/405_case.mat to /mnt/bes/pcm/tmp/scenario_405/case.mat
-->  Setting status=prepared in execute table on server


In [7]:
resp = scenario.launch_simulation()

--> Launching simulation on container


In [10]:
scenario.check_progress()

{'errors': [],
 'output': ['Validation complete!',
  'Launching scenario with parameters:',
  "{'num_segments': 1, 'interval': 24, 'n_interval': 31, 'start_index': 1, 'input_dir': '/mnt/bes/pcm/tmp/scenario_405', 'execute_dir': '/mnt/bes/pcm/tmp/scenario_405/output', 'threads': None, 'julia_env': None, 'solver_kwargs': None}"],
 'scenario_id': 405,
 'status': 'running'}

In [14]:
scenario.check_progress()

{'errors': [],
 'output': ['Validation complete!',
  'Launching scenario with parameters:',
  "{'num_segments': 1, 'interval': 24, 'n_interval': 31, 'start_index': 1, 'input_dir': '/mnt/bes/pcm/tmp/scenario_405', 'execute_dir': '/mnt/bes/pcm/tmp/scenario_405/output', 'threads': None, 'julia_env': None, 'solver_kwargs': None}",
  'Julia exception: UndefVarError: m not defined',
  'Stacktrace:',
  '[1] run_scenario_gurobi(; solver_kwargs::Nothing, kwargs::Base.Iterators.Pairs{Symbol,Any,NTuple{7,Symbol},NamedTuple{(:interval, :n_interval, :start_index, :inputfolder, :outputfolder, :threads, :num_segments),Tuple{Int64,Int64,Int64,String,String,Nothing,Int64}}}) at /app/src/solver_specific/gurobi.jl:12',
  '[2] (::PyCall.var"#f_kw_closure#59"{typeof(REISE.run_scenario_gurobi),Tuple{},Array{Tuple{Symbol,Any},1}})() at /root/.julia/packages/PyCall/zqDXB/src/callback.jl:36',
  '[3] _pyjlwrap_call(::Function, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /root/.julia/package

### Retrieve simulation output again

In [ ]:
from powersimdata.scenario.scenario import Scenario
scenario = Scenario("training") 
scenario.state.print_scenario_info() 
ct = scenario.state.get_ct() 
grid = scenario.state.get_grid()
congu = scenario.state.get_congu()                                                     
congl = scenario.state.get_congl()
congu_yearly = (congu.sum(axis = 0)[congu.sum(axis = 0)!=0]).sort_values(ascending=False)
congl_yearly = (congl.sum(axis = 0)[congl.sum(axis = 0)!=0]).sort_values(ascending=False)
congl[68928]
congl[68928].astype(bool).sum(axis=0)